In [ ]:
# This notebook is used to filter the CAPEs data based on the department and metrics we wanted to focus on.
# After filtration, we take the average of each of the metrics for each course over the past 5 years.
# We wrote the filtration part of this notebook from scratch, and used ChatGPT to write the functions for the averaging part.

In [ ]:
import pandas as pd
import re

In [ ]:
capes = pd.read_csv('capes_data.csv')

In [3]:
filtered_capes = capes.drop(['Total Enrolled in Course','Total CAPEs Given','Evalulation URL'], axis=1)

In [4]:
depts = ['BENG','CSE','DSC','ECE','LIGN','MAE','PSYC']

In [5]:
filtered_capes = filtered_capes[
    (filtered_capes['Quarter'].str[2:4].astype(int) > 18) & 
    (filtered_capes['Course'].str.split(' ').str[0].isin(depts))
]

In [6]:
filtered_capes['Course']=filtered_capes['Course'].str.split(' - ').str[0]

In [10]:
filtered_capes['Quarter'].str[0:2].unique()

array(['SP', 'WI', 'FA', 'S3', 'S2', 'S1'], dtype=object)

In [60]:
filtered_capes.to_csv('filtered_capes_data.csv', index=False)

In [ ]:
def extract_numeric_grade(grade):
    """
    Extract the numerical part of a grade in the format 'B (3.03)'.
    Returns None if the input is invalid or empty.
    """
    if pd.isna(grade):
        return None
    match = re.search(r'\(([\d.]+)\)', grade)  # Look for a number inside parentheses
    return float(match.group(1)) if match else None

def summarize_capes_data_with_top_professor(file_path):
    # Load the CAPES data
    df = pd.read_csv(file_path)

    # Ensure relevant columns are present
    required_columns = [
        "Course",
        "Instructor",
        "Percentage Recommended Professor",
        "Percentage Recommended Class",
        "Study Hours per Week",
        "Average Grade Expected",
        "Average Grade Received",
        "Quarter"  # Ensure "Quarter" column is present
    ]
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns in CAPES data: {missing_columns}")

    # Clean and process numerical fields
    df["Percentage Recommended Class"] = df["Percentage Recommended Class"].str.rstrip('%').astype(float)
    df["Percentage Recommended Professor"] = df["Percentage Recommended Professor"].str.rstrip('%').astype(float)
    df["Study Hours per Week"] = pd.to_numeric(df["Study Hours per Week"], errors="coerce")
    df["Average Grade Expected"] = df["Average Grade Expected"].apply(extract_numeric_grade)
    df["Average Grade Received"] = df["Average Grade Received"].apply(extract_numeric_grade)

    # Calculate the average rating for each professor for each course
    professor_averages = df.groupby(["Course", "Instructor"]).agg(
        average_recommended_professor=("Percentage Recommended Professor", "mean")
    ).reset_index()

    # Identify the top professor for each course
    top_professors = professor_averages.loc[
        professor_averages.groupby("Course")["average_recommended_professor"].idxmax()
    ].rename(columns={
        "Instructor": "Top Professor",
        "average_recommended_professor": "Top Professor Rating"
    })

    # Group by unique Course and compute overall averages
    summary = df.groupby("Course").agg(
        average_recommended_class=("Percentage Recommended Class", "mean"),
        average_study_hours=("Study Hours per Week", "mean"),
        average_grade_expected=("Average Grade Expected", "mean"),
        average_grade_received=("Average Grade Received", "mean")
    ).reset_index()

    # Extract quarters without the year and aggregate unique values
    df["Quarter_Only"] = df["Quarter"].str[:2]  # Extract the first two characters (e.g., "WI", "FA")
    quarters_offered = df.groupby("Course")["Quarter_Only"].apply(lambda x: ', '.join(sorted(set(x)))).reset_index()
    quarters_offered.rename(columns={"Quarter_Only": "Quarters Offered"}, inplace=True)

    # Merge quarters information into the summary
    summary = summary.merge(quarters_offered, on="Course")

    # Merge top professor information into the summary
    summary = summary.merge(top_professors, on="Course")

    # Round numeric values for cleaner output
    summary = summary.round(2)

    return summary


In [19]:
# Path to your CAPES CSV file
file_path = "filtered_capes_data.csv"

# Summarize CAPES data
summary = summarize_capes_data_with_top_professor(file_path)

In [20]:
# Display the summary
summary.rename(columns={"Course":"C","average_recommended_class": "R", "average_study_hours": "H",'average_grade_expected':'GE','average_grade_received':'GR','Top Professor':'P','Top Professor Rating':'PR', 'Quarters Offered':'Q'},inplace=True)

summary = summary[summary['GR'].notna()]
# Save the summarized data to a new CSV file
summary.to_csv("average_capes_data.csv", index=False)

In [18]:
summary

,C,R,H,GE,GR,Q,P,PR
1,BENG 100,69.92,6.26,3.36,3.35,"FA, S1, SP",Abbasi Shaghayegh,100.00
2,BENG 102,68.55,6.06,3.71,3.64,SP,Engler Adam J.,72.00
3,BENG 103B,74.00,9.51,3.31,3.27,SP,Fraley Stephanie I.,77.12
4,BENG 110,79.68,6.91,3.32,3.19,"FA, WI",Mc Culloch Andrew Douglas,80.54
5,BENG 112A,93.77,6.52,3.35,3.06,WI,Valdez-Jasso Daniela,99.63
...,...,...,...,...,...,...,...,...
442,PSYC 60,89.52,5.44,3.20,2.86,"FA, S1, S2, SP, WI",Lowe Angela Beth,96.92
443,PSYC 7,97.57,3.64,3.51,3.54,"FA, S2, SP",Hanes Esther,100.00
444,PSYC 70,93.49,5.22,3.35,3.07,"FA, S1, S2, SP, WI",Lowe Angela Beth,100.00
445,PSYC 71,88.72,5.92,3.29,3.17,"FA, S1, SP, WI",Pilegard Celeste Cristine,98.04
